# Convolutional Neural Networks (CNNs) and Their Application to RF Signal Analysis

## 1. Introduction

Convolutional Neural Networks (CNNs) are a class of deep learning models designed to automatically and adaptively learn spatial or temporal features from structured data. While widely used in image processing, CNNs are also effective for sequential data, including **radio frequency (RF) signals**, due to their ability to capture local dependencies and hierarchical patterns.

---

## 2. What is a CNN?

A CNN is composed of several types of layers:

1. **Convolutional Layers**  
   - Apply learnable filters (kernels) across the input.  
   - Each filter extracts local features, e.g., edges in images or patterns in RF waveforms.  
   - Convolution preserves the local structure of the data.

2. **Activation Functions**  
   - Non-linear functions like ReLU (`f(x)=max(0,x)`) applied after convolution.  
   - Introduce non-linearity, allowing the network to model complex patterns.

3. **Pooling Layers**  
   - Reduce the dimensionality of feature maps, typically via max or average pooling.  
   - Retain dominant features and reduce computational complexity.

4. **Fully Connected Layers**  
   - Flatten the feature maps and perform classification or regression.  
   - Combine features to produce the final output (e.g., modulation type or signal class).

5. **Optional Layers**  
   - Dropout, batch normalisation, and residual connections for regularisation and training stability.

---

## 3. Why CNNs for RF Signals?

RF signals are typically represented as **time-series data**, often in the form of **I/Q samples** (in-phase and quadrature components). CNNs can extract meaningful features from these signals for tasks such as:

- **Modulation recognition**: Determining the modulation scheme (e.g., BPSK, QPSK, 16-QAM).  
- **Signal classification**: Identifying signal type or source (e.g., WiFi, LTE, radar).  
- **Anomaly detection**: Detecting interference or unusual RF behaviour.  

### Advantages of CNNs for RF:
- Local correlations: Convolutional filters detect patterns in small windows of the waveform.  
- Parameter efficiency: Shared weights reduce the number of parameters compared to fully connected networks.  
- Hierarchical features: Low-level features (oscillations) combine into higher-level patterns (modulation shapes).  

---

## 4. Input Representation

RF signals can be fed into a CNN in various forms:

1. **Raw I/Q Time-Series**
   - Shape: `(batch, 2, seq_len)` where 2 = I/Q channels.  
   - Preserves phase and amplitude information.

2. **Spectrograms or Time-Frequency Representations**
   - Compute Short-Time Fourier Transform (STFT) to create a 2D representation.  
   - Useful when frequency content is critical.  
   - Can leverage 2D CNNs designed for images.

---

## 5. CNN Architecture for RF Signals

A typical CNN for RF signal classification includes:

1. **1D Convolution Layers**  
   - Detect patterns over small segments of the waveform.  
   - Example: `Conv1d(in_channels=2, out_channels=16, kernel_size=5)`

2. **Pooling Layers**  
   - Reduce sequence length, keeping dominant features.

3. **Fully Connected Layers**  
   - Combine features to classify signals into desired categories.

4. **Output Layer**  
   - Typically a softmax layer for classification tasks.

![CNN Architecture for RF Signal Classification](https://www.researchgate.net/profile/Qingjie-Yang/publication/361255771/figure/fig3/AS:1124262041185280@1656041345680/The-architecture-of-the-CNN-used-for-RF-signal-classification-Three-convolutional-layers.png)

*Figure: CNN Architecture for RF Signal Classification. Three convolutional layers apply 32, 64, and 128 filters of size 3x3x3 pixels. Max-pooling reduces the sizes of the features by applying a local 2x3x2 maximum function. The final features are flattened and run through a fully connected layer before applying a SoftMax function. The final output consists of probabilities of being in one of the 17 classes of the database's UAVs.*

---

## 6. Example Applications

| Task | Input Type | CNN Approach |
|------|------------|--------------|
| Modulation Recognition | Raw I/Q samples | 1D CNN with convolution + pooling + fully connected layers |
| Signal Source Classification | Time-frequency spectrogram | 2D CNN on spectrogram images |
| Interference Detection | Raw I/Q or spectrum | CNN autoencoder or classifier for anomaly detection |

---

## 7. Summary

CNNs are a powerful tool for RF signal analysis because they:

- Automatically learn features from raw data.
- Capture local temporal dependencies in RF waveforms.
- Scale efficiently to large datasets.
- Can be applied to tasks such as modulation classification, signal detection, and interference identification.

By representing RF signals appropriately (time-series or spectrogram) and designing suitable CNN architectures, practitioners can achieve high-performance classification and analysis without manual feature engineering.

---

**References**

1. O'Shea, T. J., & Hoydis, J. (2017). *An Introduction to Deep Learning for the Physical Layer*. IEEE Transactions on Cognitive Communications and Networking, 3(4), 563–575.  
2. Zhang, X., et al. (2018). *Convolutional Neural Networks for Modulation Recognition in Wireless Communications*. IEEE Wireless Communications Letters, 7(5), 862–865.  
3. Goodfellow, I., Bengio, Y., & Courville, A. (2016). *Deep Learning*. MIT Press.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# =============================================================================
# RFConvNet
# =============================================================================
# A bare-bones 1D convolutional neural network for RF signal classification.
# Input: I/Q signal samples as (batch, 2, seq_len)
# Output: Logits for classification into num_classes categories
# =============================================================================
class RFConvNet(nn.Module):
    """
    ## RFConvNet
    1D CNN for RF signal analysis.

    ### Arguments:
    - num_classes (int): Number of output classes.
    - seq_len (int): Length of input sequence (number of time steps).
    """
    def __init__(self, num_classes=4, seq_len=128):
        super(RFConvNet, self).__init__()
        # First convolution: 2 input channels (I/Q), 16 output channels
        self.conv1 = nn.Conv1d(2, 16, 5, padding=2)
        # Second convolution: 16->32 channels
        self.conv2 = nn.Conv1d(16, 32, 5, padding=2)
        # Max pooling reduces sequence length by half
        self.pool = nn.MaxPool1d(2)
        # Fully connected layers
        self.fc1 = nn.Linear(32 * (seq_len // 4), 128)
        self.fc2 = nn.Linear(128, num_classes)

    # -------------------------------------------------------------------------
    # forward
    # -------------------------------------------------------------------------
    # Computes the forward pass of the network.
    #
    # Input:
    # - x (Tensor): shape (batch, 2, seq_len)
    #
    # Output:
    # - Tensor: raw logits for each class
    # -------------------------------------------------------------------------
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # flatten for fully connected layers
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

In [ ]:
# =============================================================================
# generate_dummy_dataset
# =============================================================================
# Creates a dummy I/Q dataset for training/testing purposes.
# Replace with real RF dataset when available.
# =============================================================================
def generate_dummy_dataset(num_samples=500, seq_len=128, num_classes=4, batch_size=32):
    """
    ## generate_dummy_dataset
    Returns train and validation DataLoaders with random RF-like data.

    ### Arguments:
    - num_samples (int): Total number of samples.
    - seq_len (int): Sequence length of each I/Q sample.
    - num_classes (int): Number of output classes.
    - batch_size (int): Batch size for DataLoader.

    ### Returns:
    - train_loader, val_loader (DataLoader, DataLoader)
    """
    X = torch.randn(num_samples, 2, seq_len)
    y = torch.randint(0, num_classes, (num_samples,))
    dataset = TensorDataset(X, y)
    train_size = int(0.8 * num_samples)
    val_size = num_samples - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    return train_loader, val_loader

In [ ]:
# =============================================================================
# train_and_validate
# =============================================================================
# Performs training and validation of the RF CNN.
# Tracks loss, accuracy, confusion matrix, and classification report.
# =============================================================================
def train_and_validate(model, train_loader, val_loader, epochs=5, lr=1e-3, device=None):
    """
    ## train_and_validate
    Trains the model and evaluates it on a validation set.

    ### Arguments:
    - model (nn.Module): RFConvNet instance
    - train_loader (DataLoader): Training data loader
    - val_loader (DataLoader): Validation data loader
    - epochs (int): Number of training epochs
    - lr (float): Learning rate
    - device (torch.device): CPU or CUDA device

    ### Returns:
    - None
    """
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        # --- Training phase ---
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss /= total
        train_acc = correct / total

        # --- Validation phase ---
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                correct += predicted.eq(labels).sum().item()
                total += labels.size(0)

                all_preds.append(predicted.cpu())
                all_labels.append(labels.cpu())

        val_loss /= total
        val_acc = correct / total

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    # --- Compute metrics ---
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    print("\nConfusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds))

In [ ]:
# =============================================================================
# Example usage
# =============================================================================
if __name__ == "__main__":
    # Initialize model
    model = RFConvNet(num_classes=4, seq_len=128)
    # Generate dummy train/val data
    train_loader, val_loader = generate_dummy_dataset(num_samples=500, seq_len=128, num_classes=4, batch_size=32)
    # Train and validate
    train_and_validate(model, train_loader, val_loader, epochs=5, lr=1e-3)
